<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2_CollaborativeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

##Collaborative Filtering 

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-04-08 00:20:59--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-08 00:20:59--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-08 00:20:59--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.

In [2]:
!pip install surprise

In [3]:
import pandas as pd
import numpy as np
from surprise import KNNBasic
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split as train_test_split2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [4]:
review_df = pd.read_csv('/content/cleaned_amazon_review.zip?raw=true', compression='zip')

review_df.head()

,overall,reviewTime,reviewerID,asin,reviewText,summary,reviewTimeYear,reviewTimeMonth
0,5.0,2013-11-27,A3NHUQ33CFH3VM,1118461304,not one thing in this book seemed an obvious o...,clear on what leads to innovation,2013,11
1,5.0,2013-11-01,A3SK6VNBQDNBJE,1118461304,i have enjoyed dr alan gregerman s weekly blog...,becoming more innovative by opening yourself t...,2013,11
2,5.0,2013-10-10,A3SOFHUR27FO3K,1118461304,alan gregerman believes that innovation comes ...,the world from different perspectives,2013,10
3,5.0,2013-10-09,A1HOG1PYCAE157,1118461304,alan gregerman is a smart funny entertaining w...,strangers are your new best friends,2013,10
4,5.0,2013-09-07,A26JGAM6GZMM4V,1118461304,as i began to read this book i was again remin...,how and why it is imperative to engage learn f...,2013,9


In [5]:
#Remove all information except Reviewer ID, Product ID and rating
review_df = review_df[['reviewerID', 'asin', 'overall']]

In [6]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,NaN,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...
2,Appliances Parts & Accessories,NaN,the cigar moments of pleasure,the cigar book,NaN,amazon home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,NaN,tools home improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...


In [7]:
#Remove all information except Product ID and title
product_df = product_df[['asin']]

In [8]:
merged_df = review_df.merge(product_df, on='asin', how='inner')
merged_df.head()

,reviewerID,asin,overall
0,A3SHVDMM83IHJ4,B00002N7IL,5.0
1,A3TIWHNJXMSIU7,B00002N7IL,5.0
2,AVP16JFIT6LPL,B00002N7IL,5.0
3,A2J2T9LX2HC947,B00002N7IL,5.0
4,AJN26F4KMWD59,B00002N7IL,5.0


In [32]:
merged_df = merged_df.groupby("reviewerID").filter(lambda x: len(x) > 4)

In [33]:
merged_df.shape

(12196, 3)

In [34]:
#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = merged_df.copy()
y = merged_df['reviewerID']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [35]:
#Function that computes the RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline(reviewer_id, product_id):
    return 3.0

In [36]:
def score(cf_model):
    
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['reviewerID'], X_test['asin'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(reviewer, product) for (reviewer, product) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['overall'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [37]:
score(baseline)

1.8130899266079477

# Ratings Matrix

In [15]:
r_matrix = X_train.pivot_table(values='overall', index='reviewerID', columns='asin')

r_matrix

asin,9792954481,B00002N7HY,B00002N7IL,B00004SQHD,B00004YWK2,B00005OU6T,B00006IV17,B00007E7RZ,B00009W3GY,B00009W3HB,...,B01H9WQ9QA,B01HAA3L9Y,B01HBPGP28,B01HDXLRGC,B01HDXM0UE,B01HDZ4OOM,B01HHBKZTA,B01HHF1G5I,B01HHF1HQQ,B01HICGMWC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10CGYG65ZQDNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10GYWIFBQU7C6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10I4NC8TY5NMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10IHLG5RZHRMV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZGJF9K92NXJS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZJK0A7AZTSN8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZJRLTZ0OVYG9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(reviewer_id, product_id):
    
    #Check if movie_id exists in r_matrix
    if product_id in r_matrix:
        #Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[product_id].mean()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0
    
    return mean_rating

In [17]:
#Compute RMSE for the Mean model
score(cf_user_mean)

1.2412637796442219

## Weighted Mean

In [18]:
r_matrix_dummy = r_matrix.copy().fillna(0)

In [19]:
r_matrix_dummy

asin,9792954481,B00002N7HY,B00002N7IL,B00004SQHD,B00004YWK2,B00005OU6T,B00006IV17,B00007E7RZ,B00009W3GY,B00009W3HB,...,B01H9WQ9QA,B01HAA3L9Y,B01HBPGP28,B01HDXLRGC,B01HDXM0UE,B01HDZ4OOM,B01HHBKZTA,B01HHF1G5I,B01HHF1HQQ,B01HICGMWC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10CGYG65ZQDNE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10GYWIFBQU7C6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10I4NC8TY5NMM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10IHLG5RZHRMV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZGJF9K92NXJS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZJK0A7AZTSN8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZJRLTZ0OVYG9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [21]:
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim

reviewerID,A100WO06OQR8BQ,A10CGYG65ZQDNE,A10GYWIFBQU7C6,A10I4NC8TY5NMM,A10IHLG5RZHRMV,A10PEXB6XAQ5XF,A10PJB4MA8ITZO,A10Y058K7B96C6,A10ZBR6O8S8OCY,A111W73TEATLBS,...,AZ3AI1RNUX3YB,AZ5WZ0KJKPFLC,AZ8F7N52QBHQ3,AZAFVSVXJETPO,AZCOWCNGA721W,AZGJF9K92NXJS,AZJK0A7AZTSN8,AZJRLTZ0OVYG9,AZKFIZ5IESLW3,AZKPBWYY14GK1
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,1.0,0.0,0.0,0.0,0.0,0.0,0.217597,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10CGYG65ZQDNE,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10GYWIFBQU7C6,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10I4NC8TY5NMM,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10IHLG5RZHRMV,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZGJF9K92NXJS,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AZJK0A7AZTSN8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AZJRLTZ0OVYG9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.153243,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(reviewer_id, product_id):
    
    #Check if movie_id exists in r_matrix
    if product_id in r_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[reviewer_id]
        
        #Get the user ratings for the product in question
        m_ratings = r_matrix[product_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        if sim_scores.sum() == 0.0:
          wmean_rating = 3.0
        else:
          wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    return wmean_rating

In [23]:
score(cf_user_wmean)

1.5072479560642944

## KNN

In [24]:
merged_df

,reviewerID,asin,overall
2,AVP16JFIT6LPL,B00002N7IL,5.0
30,A11QC8XY4UEWI7,B00004SQHD,5.0
105,A21YS2PSI5APWP,B00004YWK2,5.0
159,A2MVWBVM0CIAQ4,B00004YWK2,5.0
201,ALXRKQ572JVN4,B00004YWK2,5.0
...,...,...,...
601450,A3BWAUC9HZ90HP,B01HAA3L9Y,4.0
601456,A187ZYXJND15XH,B01HAA3L9Y,5.0
601505,A2P9QU3Y6B765N,B01HDXM0UE,5.0
601514,A1YHDVQBFA2I9Y,B01HDXLRGC,5.0


In [43]:
#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

#Create the dataset to be used for building the filter

data = Dataset.load_from_df(merged_df, reader)

#Define the algorithm object; in this case kNN
knn = KNNBasic()

#Evaluate the performance in terms of RMSE
cross_validate(knn, data, measures=['RMSE'], cv=10, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0221  0.9740  0.9308  0.9910  0.9596  0.9547  0.9098  0.9783  1.0001  0.9571  0.

{'fit_time': (0.17089080810546875,
  0.17902088165283203,
  0.15864062309265137,
  0.17263007164001465,
  0.16673970222473145,
  0.16298127174377441,
  0.17560911178588867,
  0.1592574119567871,
  0.1674659252166748,
  0.16901373863220215),
 'test_rmse': array([1.02207258, 0.97397141, 0.93080734, 0.99096228, 0.95956413,
        0.95470491, 0.90980423, 0.97827473, 1.00006293, 0.95709315]),
 'test_time': (0.1965770721435547,
  0.23206019401550293,
  0.2520782947540283,
  0.23138713836669922,
  0.21855568885803223,
  0.2377176284790039,
  0.21453452110290527,
  0.25155210494995117,
  0.22553062438964844,
  0.24195218086242676)}

In [26]:
#Define the SVD algorithm object
svd = SVD()

#Evaluate the performance in terms of RMSE
cross_validate(svd, data, measures=['RMSE'], cv=10, verbose=True)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8406  0.7934  0.7727  0.8659  0.8093  0.8502  0.8211  0.8374  0.8574  0.8746  0.8323  0.0311  
Fit time          0.64    0.68    0.69    0.67    0.66    0.67    0.69    0.65    0.66    0.65    0.67    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.03    0.02    0.02    0.01    0.01    


{'fit_time': (0.6408581733703613,
  0.678574800491333,
  0.6914792060852051,
  0.6690244674682617,
  0.6561598777770996,
  0.6667499542236328,
  0.6886096000671387,
  0.6537950038909912,
  0.6584043502807617,
  0.6541693210601807),
 'test_rmse': array([0.84059294, 0.79340847, 0.77272063, 0.86590819, 0.80931268,
        0.8501871 , 0.82113479, 0.83742962, 0.85738644, 0.87461629]),
 'test_time': (0.01024484634399414,
  0.010013818740844727,
  0.012191057205200195,
  0.010333538055419922,
  0.01060795783996582,
  0.010306358337402344,
  0.012028217315673828,
  0.02753615379333496,
  0.018162012100219727,
  0.018662691116333008)}

In [27]:
data = Dataset.load_from_df(merged_df, reader)
trainset, testset = train_test_split2(data, test_size=.2, random_state=42)

# We'll use the famous SVD algorithm.
algo = SVD()

cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8318  0.8720  0.8423  0.8337  0.8674  0.8183  0.8390  0.8385  0.8120  0.8298  0.8385  0.0180  
Fit time          0.68    0.65    0.67    0.68    0.65    0.66    0.64    0.64    0.63    0.65    0.65    0.02    
Test time         0.02    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.00    
RMSE: 0.8265


0.8264958322828347

In [39]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [35]:
benchmark = []
# Iterate over all algorithms
data = Dataset.load_from_df(merged_df, reader)
algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fa522547690>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7fa52242f450>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7fa52242f210>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7fa52242f290>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7fa52242f1d0>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7fa52364d350>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x7fa5240eb190>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x7fa5240eb690>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x7fa523585f10>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7fa51fe373d0>, <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x7fa522811090>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fa5225

In [36]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.833016,4.962692,0.128929
SVD,0.844456,0.601476,0.024619
BaselineOnly,0.876595,0.026231,0.012295
KNNBaseline,0.888221,0.162468,0.387472
KNNWithZScore,0.916629,0.230789,0.383158
KNNWithMeans,0.923341,0.161816,0.378604
SlopeOne,0.931267,0.505021,0.061088
KNNBasic,0.965961,0.137172,0.398791
CoClustering,1.002439,0.608331,0.013886


In [40]:
param_grid = {
    "n_epochs": [10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 0.845842812794116
BEST params: 	 {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [43]:
data = Dataset.load_from_df(merged_df, reader)
trainset, testset = train_test_split2(data, test_size=.2, random_state=42)

# We'll use the famous SVD algorithm.
algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])

cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8223  0.7826  0.8570  0.8752  0.8430  0.8545  0.8206  0.8476  0.8468  0.8156  0.8365  0.0251  
Fit time          0.68    0.67    0.65    0.67    0.64    0.66    0.66    0.66    0.67    0.68    0.67    0.01    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.14    0.01    0.01    0.01    0.02    0.04    
RMSE: 0.8231


0.8230556855080229

In [53]:
merged_df2 = review_df.merge(product_df, on='asin', how='inner')

In [54]:
merged_df2 = merged_df2.groupby("reviewerID").filter(lambda x: len(x) > 1)

In [55]:
merged_df2.shape

(150090, 3)

In [62]:
data = Dataset.load_from_df(merged_df2, reader)

# We'll use the famous SVD algorithm.
algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])

cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0037  1.0015  1.0052  1.0097  0.9990  1.0136  1.0100  1.0109  0.9962  1.0112  1.0061  0.0056  
Fit time          9.09    9.15    9.17    9.08    9.16    9.07    9.43    9.13    9.14    9.04    9.14    0.10    
Test time         0.21    0.14    0.14    0.15    0.14    0.14    0.16    0.14    0.17    0.14    0.15    0.02    


{'fit_time': (9.093544244766235,
  9.145359754562378,
  9.168312549591064,
  9.077564001083374,
  9.162457466125488,
  9.06969666481018,
  9.427106142044067,
  9.126383781433105,
  9.13770866394043,
  9.039802074432373),
 'test_rmse': array([1.00370144, 1.00149871, 1.00519209, 1.00973815, 0.99897859,
        1.01360805, 1.00999702, 1.01089053, 0.99618603, 1.01120724]),
 'test_time': (0.20816373825073242,
  0.13960552215576172,
  0.14464592933654785,
  0.15394139289855957,
  0.14124703407287598,
  0.1426253318786621,
  0.16323399543762207,
  0.13778996467590332,
  0.1655595302581787,
  0.14143776893615723)}

In [63]:
trainset = data.build_full_trainset()
algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])
algo.fit(trainset)

In [64]:
## SAVING TRAINED MODEL
from surprise import dump
import os
model_filename = "./model.pickle"
print (">> Starting dump")
# Dump algorithm and reload it.
file_name = os.path.expanduser(model_filename)
dump.dump(file_name, algo=algo)
print (">> Dump done")
print(model_filename)

>> Starting dump
>> Dump done
./model.pickle


In [40]:
def load_model(model_filename):
    print (">> Loading dump")
    from surprise import dump
    import os
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    print (">> Loaded dump")
    return loaded_model

In [41]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [62]:
# Define a dictionary with column A
data1 = ['A8WEXFRWX1ZHH']
	
# Define another dictionary with column B
data2 = product_df['asin'].tolist()

# Convert the dictionary into DataFrame
df = pd.DataFrame(data1)

# Convert the dictionary into DataFrame
df1 = pd.DataFrame(data2)

# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
df['key'] = 1
df1['key'] = 1

# to obtain the cross join we will merge
# on the key and drop it.
result = pd.merge(df, df1, on ='key').drop("key", 1)
result = result.rename(columns={'0_x': 'reviewerID', '0_y': 'asin'})
result['overall']=0.0
result


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,reviewerID,asin,overall
0,A8WEXFRWX1ZHH,7301113188,0.0
1,A8WEXFRWX1ZHH,7861850250,0.0
2,A8WEXFRWX1ZHH,8792559360,0.0
3,A8WEXFRWX1ZHH,9792954481,0.0
4,A8WEXFRWX1ZHH,B00002N5EL,0.0
...,...,...,...
30234,A8WEXFRWX1ZHH,B01HJH92JQ,0.0
30235,A8WEXFRWX1ZHH,B01HJH2I5G,0.0
30236,A8WEXFRWX1ZHH,B01HJH0J1G,0.0
30237,A8WEXFRWX1ZHH,B01HJHHEA0,0.0


In [63]:
model_filename = "./model.pickle"
loaded_model = load_model(model_filename)
valid_Dataset = Dataset.load_from_df(result, reader)

testset = [valid_Dataset.df.loc[i].to_list() for i in range(len(valid_Dataset.df))]
predictions = loaded_model.test(testset)
#prediction = loaded_model.predict('AVP16JFIT6LPL', 'B00002N7IL')
top_n = get_top_n(predictions, n=100)

>> Loading dump
>> Loaded dump


In [68]:
loaded_model.predict('A8WEXFRWX1ZHH','B00J4V77DO')

Prediction(uid='A8WEXFRWX1ZHH', iid='B00J4V77DO', r_ui=None, est=4.117120046035163, details={'was_impossible': False})

In [64]:
merged_df2[merged_df2['reviewerID']=='A8WEXFRWX1ZHH']

,reviewerID,asin,overall
69873,A8WEXFRWX1ZHH,B0014CN8Y8,5.0
71892,A8WEXFRWX1ZHH,B0014CN8Y8,5.0
99861,A8WEXFRWX1ZHH,B0006GVNOA,5.0
99871,A8WEXFRWX1ZHH,B0006GVNOA,5.0
99881,A8WEXFRWX1ZHH,B0006GVNOA,5.0
...,...,...,...
101871,A8WEXFRWX1ZHH,B0006GVNOA,5.0
101881,A8WEXFRWX1ZHH,B0006GVNOA,5.0
101891,A8WEXFRWX1ZHH,B0006GVNOA,5.0
570349,A8WEXFRWX1ZHH,B00IYZP4AY,4.0


In [65]:
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

A8WEXFRWX1ZHH ['B00009W3PL', 'B0002JOZNM', 'B0009H7CAU', 'B000B7VBSM', 'B000BQ8D7A', 'B000BRKYU8', 'B000FOXCJC', 'B000FP8LK6', 'B000HMBPAA', 'B000JTBMOA', 'B000NCTOUM', 'B000XSTXKW', 'B000YRA2R0', 'B00147FJM2', 'B00149JVOC', 'B0014CN8Y8', 'B0015S4KIO', 'B001CA1XKS', 'B001DPLZFK', 'B001E0K8FM', 'B001EYAU88', 'B001GDGHKC', 'B001I1BMIE', 'B001KGX5G4', 'B001LQXU0O', 'B001TJ7284', 'B0029XL7H6', 'B002C3YNBK', 'B002TSG5TG', 'B0036BX2YG', 'B003BIGDJU', 'B003BIGDEU', 'B0046C04XQ', 'B004EAA3OU', 'B004H3XV34', 'B004LYLP7I', 'B004XLDELE', 'B004XLDEAU', 'B004XLDKHM', 'B004XLDCM0', 'B004XLE3JG', 'B004XLE1TI', 'B004XLEGA2', 'B0050KJO4S', 'B0053F7UIM', 'B0053F7TQA', 'B0053F8CES', 'B0053F804A', 'B0053F80JA', 'B0053F9EKE', 'B0053F8D1U', 'B0053F8DVA', 'B0053F9IGO', 'B0053Y2KTW', 'B0053Y2ORU', 'B0053Y3A8M', 'B005459K7A', 'B005459FRK', 'B005459F8Y', 'B005459Y6W', 'B00545AZ0Q', 'B00545ASFS', 'B00545A6LY', 'B005AR8KDM', 'B005AR75A6', 'B005ARC4KW', 'B005ARC2XQ', 'B005B48M0U', 'B005B471PW', 'B005B4D98A', 'B005